In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import random
import crossval #this is the file Eli wrote



from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier, BaggingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score, plot_precision_recall_curve, precision_recall_curve, precision_score, recall_score

from sklearn.model_selection import RandomizedSearchCV, train_test_split
import shap
import lightgbm as lgb 

pd.set_option('max_colwidth', 80)

In [10]:
df = pd.read_csv('final_data.csv') 
df.head()

,TARGET_FID,Length,Install_year,Arterial_class,Surface_width,Speed_limit,Slope,Width,Break_Yr,Target,...,Surface_type_,Surface_type_AC,Surface_type_AC/AC,Surface_type_AC/PCC,Surface_type_GRAVEL,Surface_type_PCC,Surface_type_ST,AGE,Nearby_breaks_1yr,Process_year
0,1,314.79,1972,2.0,40.0,25.0,0.0,8.0,NaN,0,...,0,0,0,0,0,1,0,38,0,2010
1,2,363.39,1972,0.0,46.0,20.0,4.0,8.0,NaN,0,...,0,1,0,0,0,0,0,38,0,2010
2,3,323.51,1972,0.0,46.0,20.0,4.0,8.0,NaN,0,...,0,1,0,0,0,0,0,38,0,2010
3,4,329.13,1928,0.0,0.0,20.0,6.0,12.0,NaN,0,...,0,0,0,0,0,0,1,82,0,2010
4,5,273.64,1928,2.0,42.0,25.0,4.0,18.0,NaN,0,...,0,0,0,1,0,0,0,82,0,2010


In [3]:
param_list = {
      'n_estimators': [50, 150, 300],
      'max_depth': [5, 7, 10]
    } 
param = {}
for key in param_list.keys():
    val = random.choice(param_list[key])
    param[key] = val

In [21]:
def rand_param_search(data, param_list, classifier_fn, n_val=2, print_results=True, n_iter=5):
    """

    """
    best_score = -1.0 
    best_param = {}
    param = {}
    for i in range(n_iter):
        print(i)
        for key in param_list.keys():
            val = random.choice(param_list[key])
            param[key] = val
        print("using the following parameters:")
        print(param)
        avgs = crossval_by_year(data, classifier_fn, param, n_val, print_results)
        print(avgs[0])
        print(avgs[1])
        print(avgs[2])
        print()
    
    if avgs[2] > best_score:
        best_param = param
        best_score = avgs[2]
    
    return best_param, best_score

In [23]:
best_score, best_param = rand_param_search(df, param_list, ExtraTreesClassifier, n_iter=3)

0
using the following parameters:
{'n_estimators': 50, 'max_depth': 5}
(532473, 53)
(116686, 53)
1.0
Cross-validation iteration 1 of 11 results:
Balanced accuracy = 0.0000
Recall            = 0.0000
Precision         = 0.0000

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
(532104, 53)
(117055, 53)
1.0
Cross-validation iteration 2 of 11 results:
Balanced accuracy = 0.0000
Recall            = 0.0000
Precision         = 0.0000

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
(531785, 53)
(117374, 53)
1.0
Cross-validation iteration 3 of 11 results:
Balanced accuracy = 0.0000
Recall            = 0.0000
Precision         = 0.0000

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
(531479, 53)
(117680, 53)
1.0
Cross-validation iteration 4 of 11 res